In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as pta
import MetaTrader5 as mt5
import plotly.express as px
from scipy import stats, signal
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from scipy.signal import find_peaks
from backtesting import Strategy, Backtest
from datetime import time, datetime, timezone

# Turn off the warning
pd.options.mode.chained_assignment = None  # or 'warn' to display the warning

In [3]:
# Start the plataform with initialize()
mt5.initialize()

# User credentials

login = 51116412
password = '39sz2vL3'
server = 'ICMarketsSC-Demo'

# log in in the server account of MT5
mt5.login(login, password, server)

True

In [4]:
tickerlists = pd.DataFrame([[symbol.name, symbol.path, symbol.description] for symbol in mt5.symbols_get()], columns = ['Ticker', 'Path', 'Description'])

tickerlists['type'] = np.where(tickerlists['Path'].str.contains("forex", case=False), 'cambial',
                               np.where(tickerlists['Path'].str.contains("crypto", case=False), 'crypto',
                                        np.where(tickerlists['Path'].str.contains("indices", case=False), 'indice',
                                                 np.where(tickerlists['Path'].str.contains("commodities", case=False), 'commodity',
                                                          np.where(tickerlists['Path'].str.contains("stock", case=False), 'stock', None)
                                                         )
                                                )
                                       )
                              )
tickerlists

,Ticker,Path,Description,type
0,EURUSD,Forex\Majors\EURUSD,Euro vs US Dollar,cambial
1,GBPUSD,Forex\Majors\GBPUSD,Great Britain Pound vs US Dollar,cambial
2,USDCHF,Forex\Majors\USDCHF,US Dollar vs Swiss Franc,cambial
3,USDJPY,Forex\Majors\USDJPY,US Dollar vs Japanese Yen,cambial
4,USDCAD,Forex\Majors\USDCAD,US Dollar vs Canadian Dollar,cambial
...,...,...,...,...
2101,DOC.NYSE,Stock CFD's\NYSE\DOC.NYSE,Healthpeak Properties,stock
2102,Cotton_N4,Commodities\Softs\Cotton_N4,Cotton No 2 -July 24 CFD,commodity
2103,Cocoa_N4,Commodities\Softs\Cocoa_N4,Cocoa -July 24 CFD,commodity
2104,Coffee_N4,Commodities\Softs\Coffee_N4,Coffee C -July 24 CFD,commodity


## Binance

In [5]:
api_key = "khLDHf87MdlUFmtoSe7xTrCMCr56ihzCBU9PBcDkH32PssRVzfLKyb5UOoAxtBfy"

api_secret = "ZqHhQYOPhwqobZRPKgx4s3DbTU0gyiCq79X28IyWdfuZ0Wd8mftG9Igiqxc8IOU8"

In [6]:
from binance.client import Client
client = Client(api_key, api_secret)

In [9]:
list = client.get_all_tickers()
list = pd.DataFrame(list)

list.tail(5)

,symbol,price
2579,SAGAUSDT,5.90950000
2580,SAGABNB,0.00968500
2581,SAGAFDUSD,5.91350000
2582,SAGATRY,194.92000000
2583,USDTMXN,16.51000000


In [10]:
info = client.get_symbol_info('BNBBTC')
info

{'symbol': 'BNBBTC',
 'status': 'TRADING',
 'baseAsset': 'BNB',
 'baseAssetPrecision': 8,
 'quoteAsset': 'BTC',
 'quotePrecision': 8,
 'quoteAssetPrecision': 8,
 'baseCommissionPrecision': 8,
 'quoteCommissionPrecision': 8,
 'orderTypes': ['LIMIT',
  'LIMIT_MAKER',
  'MARKET',
  'STOP_LOSS_LIMIT',
  'TAKE_PROFIT_LIMIT'],
 'icebergAllowed': True,
 'ocoAllowed': True,
 'quoteOrderQtyMarketAllowed': True,
 'allowTrailingStop': True,
 'cancelReplaceAllowed': True,
 'isSpotTradingAllowed': True,
 'isMarginTradingAllowed': True,
 'filters': [{'filterType': 'PRICE_FILTER',
   'minPrice': '0.00000100',
   'maxPrice': '100000.00000000',
   'tickSize': '0.00000100'},
  {'filterType': 'LOT_SIZE',
   'minQty': '0.00100000',
   'maxQty': '100000.00000000',
   'stepSize': '0.00100000'},
  {'filterType': 'ICEBERG_PARTS', 'limit': 10},
  {'filterType': 'MARKET_LOT_SIZE',
   'minQty': '0.00000000',
   'maxQty': '4253.00324583',
   'stepSize': '0.00000000'},
  {'filterType': 'TRAILING_DELTA',
   'minTra